# SparkSQL - Queries SQL no Spark

Este notebook cobre:
- Views temporárias
- Queries SQL básicas e avançadas
- CTEs (Common Table Expressions)
- Window Functions em SQL
- Subqueries e correlações

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQL_Estudo") \
    .getOrCreate()

## 1. Criando Dados e Views

In [ ]:
# Dados de exemplo
funcionarios_data = [
    (1, "João", "Vendas", 5000.0, "2023-01-15", 1),
    (2, "Maria", "TI", 7500.0, "2022-06-20", 2),
    (3, "Pedro", "Vendas", 4500.0, "2023-03-10", 1),
    (4, "Ana", "RH", 6000.0, "2021-11-05", 3),
    (5, "Carlos", "TI", 8000.0, "2020-08-22", 2),
    (6, "Julia", "TI", 6500.0, "2023-07-01", 2),
]

vendas_data = [
    (1, 1, 10000.0, "2024-01-15"),
    (2, 1, 15000.0, "2024-02-20"),
    (3, 3, 8000.0, "2024-01-25"),
    (4, 1, 12000.0, "2024-03-10"),
    (5, 3, 9500.0, "2024-02-28"),
]

departamentos_data = [
    (1, "Vendas", "São Paulo"),
    (2, "TI", "Rio de Janeiro"),
    (3, "RH", "Belo Horizonte"),
]

df_funcionarios = spark.createDataFrame(
    funcionarios_data,
    ["id", "nome", "departamento", "salario", "data_contratacao", "dept_id"]
)

df_vendas = spark.createDataFrame(
    vendas_data,
    ["venda_id", "funcionario_id", "valor", "data_venda"]
)

df_departamentos = spark.createDataFrame(
    departamentos_data,
    ["dept_id", "nome_dept", "cidade"]
)

In [ ]:
# Registrando views temporárias
df_funcionarios.createOrReplaceTempView("funcionarios")
df_vendas.createOrReplaceTempView("vendas")
df_departamentos.createOrReplaceTempView("departamentos")

# View global (visível em todas as sessões)
# df_funcionarios.createOrReplaceGlobalTempView("funcionarios_global")
# Acesso: SELECT * FROM global_temp.funcionarios_global

## 2. Queries Básicas

In [ ]:
# SELECT simples
spark.sql("SELECT * FROM funcionarios").show()

In [ ]:
# SELECT com filtro e ordenação
spark.sql("""
    SELECT nome, salario, departamento
    FROM funcionarios
    WHERE salario > 5000
    ORDER BY salario DESC
""").show()

In [ ]:
# SELECT com funções
spark.sql("""
    SELECT
        nome,
        salario,
        salario * 12 AS salario_anual,
        UPPER(nome) AS nome_maiusculo,
        YEAR(data_contratacao) AS ano_contratacao
    FROM funcionarios
""").show()

## 3. Agregações

In [ ]:
spark.sql("""
    SELECT
        departamento,
        COUNT(*) AS total_funcionarios,
        SUM(salario) AS soma_salarios,
        AVG(salario) AS media_salario,
        MIN(salario) AS menor_salario,
        MAX(salario) AS maior_salario,
        ROUND(STDDEV(salario), 2) AS desvio_padrao
    FROM funcionarios
    GROUP BY departamento
    HAVING COUNT(*) > 1
    ORDER BY media_salario DESC
""").show()

## 4. Joins

In [ ]:
# Inner Join
spark.sql("""
    SELECT
        f.nome,
        f.salario,
        d.nome_dept,
        d.cidade
    FROM funcionarios f
    INNER JOIN departamentos d ON f.dept_id = d.dept_id
""").show()

In [ ]:
# Left Join com vendas e agregação
spark.sql("""
    SELECT
        f.nome,
        f.departamento,
        COALESCE(SUM(v.valor), 0) AS total_vendas,
        COUNT(v.venda_id) AS qtd_vendas
    FROM funcionarios f
    LEFT JOIN vendas v ON f.id = v.funcionario_id
    GROUP BY f.nome, f.departamento
    ORDER BY total_vendas DESC
""").show()

## 5. Subqueries

In [ ]:
# Subquery no WHERE
spark.sql("""
    SELECT nome, salario
    FROM funcionarios
    WHERE salario > (SELECT AVG(salario) FROM funcionarios)
""").show()

In [ ]:
# Subquery correlacionada - maior salário por departamento
spark.sql("""
    SELECT
        f1.nome,
        f1.departamento,
        f1.salario
    FROM funcionarios f1
    WHERE f1.salario = (
        SELECT MAX(f2.salario)
        FROM funcionarios f2
        WHERE f2.departamento = f1.departamento
    )
""").show()

## 6. CTEs (Common Table Expressions)

In [ ]:
spark.sql("""
    WITH stats_dept AS (
        SELECT
            departamento,
            AVG(salario) AS media_salario,
            COUNT(*) AS total_func
        FROM funcionarios
        GROUP BY departamento
    ),
    vendedores_ativos AS (
        SELECT DISTINCT funcionario_id
        FROM vendas
        WHERE data_venda >= '2024-01-01'
    )
    SELECT
        f.nome,
        f.departamento,
        f.salario,
        sd.media_salario AS media_dept,
        CASE 
            WHEN va.funcionario_id IS NOT NULL THEN 'Sim' 
            ELSE 'Não' 
        END AS vendeu_2024
    FROM funcionarios f
    JOIN stats_dept sd ON f.departamento = sd.departamento
    LEFT JOIN vendedores_ativos va ON f.id = va.funcionario_id
""").show()

## 7. Window Functions em SQL

In [ ]:
spark.sql("""
    SELECT
        nome,
        departamento,
        salario,
        ROW_NUMBER() OVER (PARTITION BY departamento ORDER BY salario DESC) AS row_num,
        RANK() OVER (PARTITION BY departamento ORDER BY salario DESC) AS ranking,
        DENSE_RANK() OVER (PARTITION BY departamento ORDER BY salario DESC) AS dense_ranking,
        SUM(salario) OVER (PARTITION BY departamento) AS total_dept,
        AVG(salario) OVER (PARTITION BY departamento) AS media_dept
    FROM funcionarios
""").show(truncate=False)

In [ ]:
# Lead, Lag e outros
spark.sql("""
    SELECT
        nome,
        salario,
        LEAD(salario, 1) OVER (ORDER BY salario) AS proximo_salario,
        LAG(salario, 1) OVER (ORDER BY salario) AS salario_anterior,
        FIRST_VALUE(nome) OVER (PARTITION BY departamento ORDER BY salario DESC) AS top_salario_nome,
        NTILE(4) OVER (ORDER BY salario) AS quartil
    FROM funcionarios
""").show()

In [ ]:
# Running total (soma acumulada)
spark.sql("""
    SELECT
        v.data_venda,
        v.valor,
        SUM(v.valor) OVER (
            ORDER BY v.data_venda 
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) AS running_total
    FROM vendas v
    ORDER BY v.data_venda
""").show()

## 8. CASE WHEN

In [ ]:
spark.sql("""
    SELECT
        nome,
        salario,
        CASE
            WHEN salario < 5000 THEN 'Junior'
            WHEN salario BETWEEN 5000 AND 7000 THEN 'Pleno'
            ELSE 'Senior'
        END AS nivel,
        CASE departamento
            WHEN 'TI' THEN 'Tecnologia'
            WHEN 'RH' THEN 'Recursos Humanos'
            ELSE departamento
        END AS dept_completo
    FROM funcionarios
""").show()

## 9. Funções de Data

In [ ]:
spark.sql("""
    SELECT
        nome,
        data_contratacao,
        YEAR(data_contratacao) AS ano,
        MONTH(data_contratacao) AS mes,
        DAY(data_contratacao) AS dia,
        QUARTER(data_contratacao) AS trimestre,
        DATE_FORMAT(data_contratacao, 'dd/MM/yyyy') AS data_br,
        DATEDIFF(CURRENT_DATE(), data_contratacao) AS dias_empresa,
        ADD_MONTHS(data_contratacao, 12) AS um_ano_depois,
        LAST_DAY(data_contratacao) AS ultimo_dia_mes
    FROM funcionarios
""").show()

## 10. Funções de String

In [ ]:
spark.sql("""
    SELECT
        nome,
        UPPER(nome) AS maiusculo,
        LOWER(nome) AS minusculo,
        LENGTH(nome) AS tamanho,
        CONCAT(nome, ' - ', departamento) AS nome_dept,
        SUBSTRING(nome, 1, 3) AS primeiras_letras,
        REPLACE(departamento, 'Vendas', 'Sales') AS dept_en,
        LPAD(CAST(id AS STRING), 5, '0') AS id_formatado
    FROM funcionarios
""").show()

## 11. EXISTS, IN, UNION

In [ ]:
# EXISTS - funcionários que têm vendas
spark.sql("""
    SELECT f.nome, f.departamento
    FROM funcionarios f
    WHERE EXISTS (
        SELECT 1 FROM vendas v WHERE v.funcionario_id = f.id
    )
""").show()

In [ ]:
# NOT EXISTS - funcionários sem vendas
spark.sql("""
    SELECT f.nome, f.departamento
    FROM funcionarios f
    WHERE NOT EXISTS (
        SELECT 1 FROM vendas v WHERE v.funcionario_id = f.id
    )
""").show()

In [ ]:
# UNION
spark.sql("""
    SELECT nome, departamento FROM funcionarios WHERE departamento = 'TI'
    UNION ALL
    SELECT nome, departamento FROM funcionarios WHERE salario > 6000
""").show()

# INTERSECT
spark.sql("""
    SELECT nome FROM funcionarios WHERE departamento = 'TI'
    INTERSECT
    SELECT nome FROM funcionarios WHERE salario > 6000
""").show()

## 12. Explain - Analisando Plano de Execução

In [ ]:
spark.sql("""
    SELECT f.nome, SUM(v.valor) as total
    FROM funcionarios f
    JOIN vendas v ON f.id = v.funcionario_id
    GROUP BY f.nome
""").explain(True)